In [1]:
import pygame
import sys
from pygame.locals import *

%run logic.ipynb

class SOSGameUI:
    def __init__(self, game):
        self.player_type = {'Blue': 'human', 'Red': 'human'}
        self.game = game
        self.game.player_type = self.player_type
        
        self.screen_width = 700
        self.screen_height = 700
        self.grid_size = 400
        self.start_x = 250
        self.start_y = 100
        
        self.sidebar_width = 220
        self.sidebar_padding = 20
        self.section_spacing = 30  # Space between different sections
        
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption("SOS Game")

        self.BLACK = (0, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GRAY = (200, 200, 200)
        self.LIGHT_GRAY = (230, 230, 230)  # For button hover effect
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        self.LIGHT_BLUE = (173, 216, 230)  # For section headers
        self.SECTION_HEADER_BG = (240, 240, 250)  # Light background for section headers

        self.running = True
        self.font = pygame.font.SysFont("Arial", 24)
        self.small_font = pygame.font.SysFont("Arial", 20)  # Smaller font for less important text
        self.large_font = pygame.font.SysFont("Arial", 32, bold=True)  # Larger font for game status

        self.button_width = 180
        self.button_height = 40
        self.button_margin = 15  # Space between buttons
        
        self.input_box = pygame.Rect(self.sidebar_padding, 0, 100, self.button_height)  # Y will be set in create_layout
        self.input_text = str(self.game.n)  # Initialize with current board size
        self.input_active = False
        self.buttons = []
        
        self.create_layout()
        
        if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 1000)

    def create_layout(self):
        sidebar_x = self.sidebar_padding
        current_y = 20 
        
        self.section_headers = []
        self.section_headers.append(("Player Information", current_y))
        current_y += 35
        
        current_y += 60
        
        self.section_headers.append(("Game Mode", current_y))
        current_y += 35
        
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, "Simple Mode", self.set_simple_mode)
        current_y += self.button_height + self.button_margin
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, "General Mode", self.set_general_mode)
        current_y += self.button_height + self.section_spacing
        
        self.section_headers.append(("Board Size", current_y))
        current_y += 35
        
        # Position input box and refresh button side by side, perhaps move to top as well on the right side
        self.input_box.y = current_y + 5  # Align with button
        self.input_box.width = 80
        refresh_x = self.input_box.x + self.input_box.width + 15
        self.add_button(refresh_x, current_y, 85, self.button_height, "Refresh", self.update_board_size)
        current_y += self.button_height + self.section_spacing
        
        # Symbol Assignment for S and O
        self.section_headers.append(("Symbol Assignment", current_y))
        current_y += 35
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, "Swap S/O", self.swap_roles)
        current_y += self.button_height + self.section_spacing
        
        # Player Types Section move this to bottom? or top?
        self.section_headers.append(("Player Types", current_y))
        current_y += 35
        
        # Blue player toggle buttons cpu/human
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, 
                       "Blue: Human", self.set_blue_human)
        current_y += self.button_height + 5  # Less space between related buttons
        
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, 
                       "Blue: Computer", self.set_blue_computer)
        current_y += self.button_height + 15  # More space between different player options
        
        # Red player toggle buttons
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, 
                       "Red: Human", self.set_red_human)
        current_y += self.button_height + 5  # Less space between related buttons
        
        self.add_button(sidebar_x, current_y, self.button_width, self.button_height, 
                       "Red: Computer", self.set_red_computer)
        current_y += self.button_height + self.section_spacing
        
        self.section_headers.append(("Current Turn", current_y))
        self.current_turn_y = current_y + 35
        current_y += 70

        # scores
        self.section_headers.append(("Scores", current_y))
        self.scores_y = current_y + 35
        current_y += 70

    def draw_grid(self):
        self.cell_size = self.grid_size // self.game.n
        for row in range(1, self.game.n):
            pygame.draw.line(self.screen, self.BLACK, (self.start_x + row * self.cell_size, self.start_y),
                             (self.start_x + row * self.cell_size, self.start_y + self.grid_size), 2)
            pygame.draw.line(self.screen, self.BLACK, (self.start_x, self.start_y + row * self.cell_size),
                             (self.start_x + self.grid_size, self.start_y + row * self.cell_size), 2)
        pygame.draw.rect(self.screen, self.BLACK, (self.start_x, self.start_y, self.grid_size, self.grid_size), 2)

    def draw_game_status(self):
        if self.game.game_over:
            status_rect = pygame.Rect(self.start_x, self.start_y + self.grid_size + 20, 
                                     self.grid_size, 40)
            pygame.draw.rect(self.screen, self.LIGHT_GRAY, status_rect)
            pygame.draw.rect(self.screen, self.BLACK, status_rect, 2)
            
            if self.game.winner == 'Draw':
                text = "It's a draw!"
                color = self.BLACK
            else:
                text = f"{self.game.winner} wins!"
                color = self.RED if self.game.winner == 'Red' else self.BLUE
            
            game_over_text = self.large_font.render(text, True, color)
            text_x = self.start_x + (self.grid_size - game_over_text.get_width()) // 2
            text_y = self.start_y + self.grid_size + 25
            self.screen.blit(game_over_text, (text_x, text_y))

    def draw_sidebar(self):
        pygame.draw.rect(self.screen, self.GRAY, (0, 0, self.sidebar_width, self.screen_height))
        
        for header_text, y_pos in self.section_headers:

            header_rect = pygame.Rect(0, y_pos - 5, self.sidebar_width, 30)
            pygame.draw.rect(self.screen, self.SECTION_HEADER_BG, header_rect)
            pygame.draw.line(self.screen, self.LIGHT_BLUE, (0, y_pos + 25), (self.sidebar_width, y_pos + 25), 2)

            self.draw_text(header_text, self.sidebar_padding, y_pos, self.BLACK, bold=True)
        
        blue_label = f"Blue: {self.player_type['Blue'].capitalize()} ({self.game.player_symbols['Blue']})"
        red_label = f"Red: {self.player_type['Red'].capitalize()} ({self.game.player_symbols['Red']})"
        self.draw_text(blue_label, self.sidebar_padding, self.section_headers[0][1] + 35, self.BLUE)
        self.draw_text(red_label, self.sidebar_padding, self.section_headers[0][1] + 65, self.RED)
        
        pygame.draw.rect(self.screen, self.WHITE, self.input_box)
        pygame.draw.rect(self.screen, self.BLACK, self.input_box, 2)
        text_surface = self.font.render(self.input_text, True, self.BLACK)
        self.screen.blit(text_surface, (self.input_box.x + 5, self.input_box.y + 5))
        
        color = self.BLUE if self.game.current_player == 'Blue' else self.RED
        self.draw_text(f"{self.game.current_player} ({self.game.player_symbols[self.game.current_player]})", 
                      self.sidebar_padding, self.current_turn_y, color)
        
        if self.game.mode == "General":
            if hasattr(self.game, 'blue_score') and hasattr(self.game, 'red_score'):

                score_rect = pygame.Rect(self.sidebar_padding - 5, self.scores_y - 5,
                                        self.sidebar_width - 2*self.sidebar_padding + 10, 40)
                pygame.draw.rect(self.screen, self.WHITE, score_rect)
                pygame.draw.rect(self.screen, self.BLACK, score_rect, 2)
                
                # Draw the scores
                blue_score = f"Blue: {self.game.blue_score}"
                red_score = f"Red: {self.game.red_score}"
                self.draw_text(blue_score, self.sidebar_padding + 5, self.scores_y, self.BLUE)
                self.draw_text(red_score, self.sidebar_padding + 100, self.scores_y, self.RED)

        mouse_pos = pygame.mouse.get_pos()
        for rect, text, action in self.buttons:
            is_active_button = False
            if text == "Blue: Human" and self.player_type['Blue'] == 'human':
                is_active_button = True
            elif text == "Blue: Computer" and self.player_type['Blue'] == 'computer':
                is_active_button = True
            elif text == "Red: Human" and self.player_type['Red'] == 'human':
                is_active_button = True
            elif text == "Red: Computer" and self.player_type['Red'] == 'computer':
                is_active_button = True
            elif text == "Simple Mode" and self.game.mode == "Simple":
                is_active_button = True
            elif text == "General Mode" and self.game.mode == "General":
                is_active_button = True
            
            if is_active_button:
                pygame.draw.rect(self.screen, self.LIGHT_BLUE, rect)
            elif rect.collidepoint(mouse_pos):
                pygame.draw.rect(self.screen, self.LIGHT_GRAY, rect)
            else:
                pygame.draw.rect(self.screen, self.WHITE, rect)
            
            pygame.draw.rect(self.screen, self.BLACK, rect, 2)
            label = self.font.render(text, True, self.BLACK)
            self.screen.blit(label, (rect.x + (rect.width - label.get_width()) // 2, rect.y + (rect.height - label.get_height()) // 2))

    def add_button(self, x, y, width, height, text, action):

        rect = pygame.Rect(x, y, width, height)
        self.buttons.append((rect, text, action))

    def draw_text(self, text, x, y, color, bold=False):
        if bold:
            temp_font = pygame.font.SysFont("Arial", 24, bold=True)
            label = temp_font.render(text, True, color)
        else:
            label = self.font.render(text, True, color)
        self.screen.blit(label, (x, y))

    def draw_board(self):
        self.cell_size = self.grid_size // self.game.n

        for row in range(self.game.n):
            for col in range(self.game.n):
                cell = self.game.board[row][col]
                if cell:
                    symbol, player = cell
                    color = self.BLUE if player == 'Blue' else self.RED
                    text = self.font.render(symbol, True, color)
                    x = self.start_x + col * self.cell_size + (self.cell_size - text.get_width()) // 2
                    y = self.start_y + row * self.cell_size + (self.cell_size - text.get_height()) // 2
                    self.screen.blit(text, (x, y))

    def update_board_size(self):
        if self.input_text.isdigit():
            size = int(self.input_text)
            if size > 2:
                new_game = SOSGame(size, self.game.mode, player_type=self.player_type)
                new_game.player_type = self.player_type  # Ensure player types are preserved
                self.game = new_game
                self.refresh_ui()
        if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 1000)

    def refresh_ui(self):
        self.screen.fill(self.WHITE)
        self.cell_size = self.grid_size // self.game.n
        self.draw_sidebar()
        self.draw_grid()
        self.draw_board()
        self.draw_game_status()  # Draw game status below the grid
        pygame.display.flip()

    # New explicit player type setter methods
    def set_blue_human(self):
        self.player_type['Blue'] = 'human'
        self.game.player_type = self.player_type
        self.refresh_ui()
        # Stop timer if it's this player's turn and they're now human
        if self.game.current_player == 'Blue':
            pygame.time.set_timer(pygame.USEREVENT, 0)

    def set_blue_computer(self):
        self.player_type['Blue'] = 'computer'
        self.game.player_type = self.player_type
        self.refresh_ui()
        # Start timer if it's this player's turn and they're now computer
        if self.game.current_player == 'Blue' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 1000)

    def set_red_human(self):
        self.player_type['Red'] = 'human'
        self.game.player_type = self.player_type
        self.refresh_ui()
        # Stop timer if it's this player's turn and they're now human
        if self.game.current_player == 'Red':
            pygame.time.set_timer(pygame.USEREVENT, 0)

    def set_red_computer(self):
        self.player_type['Red'] = 'computer'
        self.game.player_type = self.player_type
        self.refresh_ui()
        # Start timer if it's this player's turn and they're now computer
        if self.game.current_player == 'Red' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 1000)

    def set_simple_mode(self):
        new_game = SOSGame(self.game.n, mode="Simple", player_type=self.player_type)
        new_game.player_type = self.player_type
        self.game = new_game
        
        if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 200)
        else:
            pygame.time.set_timer(pygame.USEREVENT, 0)
            
        self.refresh_ui()

    def set_general_mode(self):
        new_game = SOSGame(self.game.n, mode="General", player_type=self.player_type)
        new_game.player_type = self.player_type
        new_game.blue_score = 0
        new_game.red_score = 0
        self.game = new_game
        
        if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
            pygame.time.set_timer(pygame.USEREVENT, 200)
        else:
            pygame.time.set_timer(pygame.USEREVENT, 0)
            
        self.refresh_ui()

    def swap_roles(self):
        self.game.swap_roles()
        self.refresh_ui()

    def handle_mouse_click(self, pos):
        if self.input_box.collidepoint(pos):
            self.input_active = True
        else:
            self.input_active = False

        for rect, _, action in self.buttons:
            if rect.collidepoint(pos):
                action()
                return

        if self.game.game_over:
            return

        if (self.start_x <= pos[0] <= self.start_x + self.grid_size and
            self.start_y <= pos[1] <= self.start_y + self.grid_size):
            col = (pos[0] - self.start_x) // self.cell_size
            row = (pos[1] - self.start_y) // self.cell_size
            
            if 0 <= row < self.game.n and 0 <= col < self.game.n:
                self.game.place_symbol(row, col)
                self.refresh_ui()
                
                if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
                    pygame.time.set_timer(pygame.USEREVENT, 1000)
                else:
                    pygame.time.set_timer(pygame.USEREVENT, 0)

    def run(self):
        while self.running:
            self.screen.fill(self.WHITE)
            self.draw_sidebar()
            self.draw_grid()
            self.draw_board()
            self.draw_game_status()  # Draw game status below the grid

            for event in pygame.event.get():
                if event.type == pygame.USEREVENT:
                    self.game.place_symbol(None, None)  # Computer makes a move
                    self.refresh_ui()
                    
                    if self.game.player_type[self.game.current_player] == 'computer' and not self.game.game_over:
                        pygame.time.set_timer(pygame.USEREVENT, 1000)
                    else:
                        pygame.time.set_timer(pygame.USEREVENT, 0)
                    continue

                elif event.type == pygame.QUIT:
                    self.running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    self.handle_mouse_click(event.pos)
                elif event.type == pygame.KEYDOWN:
                    if self.input_active:
                        if event.key == pygame.K_RETURN:
                            self.update_board_size()
                            self.input_active = False
                        elif event.key == pygame.K_BACKSPACE:
                            self.input_text = self.input_text[:-1]
                        elif event.unicode.isdigit():
                            self.input_text += event.unicode

            pygame.display.flip()

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    pygame.init()
    if not pygame.font.get_init():
        pygame.font.init()
    game = SOSGame()
    ui = SOSGameUI(game)
    ui.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
